In [ ]:
#!pip install eventregistry

In [ ]:
from eventregistry import * # type: ignore
import pandas as pd
import json

# Load the API key from the JSON file
with open("config.json", "r") as file:
    config = json.load(file)
api_key = config["api_key"]

# Initialize EventRegistry with your API key and limit search to the last month
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False) # type: ignore

# Define topics to search for
topics = [
    "Donald Trump", "Kamala Harris", "Israel", "Palestine", "Palestinians", "Hamas", "FEMA", "Abortion",
    "Inflation", "Unemployment", "Economy", "Dockworkers", "ILA Port"
]

# List to store the names of all generated DataFrames
dataframe_names = []

# Loop through each topic, fetch articles, create a DataFrame, and save to CSV
for topic in topics:
    # Get the URI for the concept
    concept_uri = er.getConceptUri(topic)
    
    # Define the query for each topic
    q = QueryArticlesIter( # type: ignore
        conceptUri=concept_uri,
        sourceLocationUri=er.getLocationUri("United States"),  # Only US sources
    )

    # List to hold each article's data as a dictionary
    articles_data = []

    # Fetch and process articles
    for art in q.execQuery(er, sortBy="date", maxItems=500):
        articles_data.append({
            "title": art.get("title"),
            "source": art.get("source", {}).get("title"),
            "author": art.get("author"),
            "url": art.get("url"),
            "publishedAt": art.get("dateTime"),
            "content": art.get("body")
        })

    # Create a DataFrame from the list of dictionaries
    articles_df = pd.DataFrame(articles_data)
    
    # Save the DataFrame to a CSV file
    file_name = f"{topic.replace(' ', '_')}_articles.csv"
    articles_df.to_csv(file_name, index=False)

    # Dynamically set the DataFrame name based on the topic, replacing spaces with underscores
    df_name = f"{topic.replace(' ', '_')}_df"
    globals()[df_name] = articles_df

    # Append the DataFrame name to the list
    dataframe_names.append(df_name)

# Print the list of all generated DataFrame names
print("Generated DataFrames:", dataframe_names)

In [ ]:
# Example: Access the "Donald Trump" DataFrame
Donald_Trump_df.head() # type: ignore